In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import pandas as pd
import sys
sys.path.append("..")

In [39]:
# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [37]:
def one_hot(seq):
    conversion_dict={
        'A':np.array([1,0,0,0]),
        'C':np.array([0,1,0,0]),
        'G':np.array([0,0,1,0]),
        'T':np.array([0,0,0,1])
    }
    enc_arr=conversion_dict[seq[0]]
    for i in seq[1:]:
        enc_arr=np.vstack((enc_arr,conversion_dict[i]))
    return torch.from_numpy(enc_arr.T)
        
    

In [38]:
testseq="AGACTAGCAG"
one_hot(testseq)


tensor([[1, 0, 1, 0, 0, 1, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [3]:

# load data
dataset1=pd.read_csv('data/random_train_pc.csv')

class fiveprimeutrs(Dataset):
    def __init__(self, seqs, mrl):
        self.seqs = seqs
        self.mrl = mrl
    def __len__(self):
        return len(self.seqs)
    def __getitem__(self, idx):
        seq = self.seqs[idx]
        mrl = self.mrl[idx]
        

In [5]:
#Model
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 120, kernel_size = (8,4))
        self.conv2 = nn.Conv2d(120, 120, kernel_size = (8,1))
        self.conv3 = nn.Conv2d(120, 120, kernel_size = (8,1))
        self.fc1 = nn.Linear(120, 40)
        self.drop1 = nn.Dropout(0.2)
        self.out = nn.Linear(640, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = self.out(x)
        return x



BCDEFGHIJKLMNOP


In [16]:
conversion_dict={
        'A':np.array([1,0,0,0]),
        'C':np.[0,1,0,0],
        'G':[0,0,1,0],
        'T':[0,0,0,1]
    }
testseq="AGACTAGCAG"
for i in testseq:
    print(i,conversion_dict[i])

A [1 0 0 0]
G [0, 0, 1, 0]
A [1 0 0 0]
C [0, 1, 0, 0]
T [0, 0, 0, 1]
A [1 0 0 0]
G [0, 0, 1, 0]
C [0, 1, 0, 0]
A [1 0 0 0]
G [0, 0, 1, 0]


In [40]:
net=Model()
net.to(device)

Model(
  (conv1): Conv2d(1, 120, kernel_size=(8, 4), stride=(1, 1))
  (conv2): Conv2d(120, 120, kernel_size=(8, 1), stride=(1, 1))
  (conv3): Conv2d(120, 120, kernel_size=(8, 1), stride=(1, 1))
  (fc1): Linear(in_features=120, out_features=40, bias=True)
  (drop1): Dropout(p=0.2, inplace=False)
  (out): Linear(in_features=640, out_features=1, bias=True)
)

In [41]:
net(one_hot(testseq))

RuntimeError: Input type (long long) and bias type (float) should be the same